# Lesson 3: Chatbot Example

In this lesson, you will familiarize yourself with the chatbot example you will work on during this course. The example includes the tool definitions and execution, as well as the chatbot code. Make sure to interact with the chatbot at the end of this notebook.

## Import Libraries

In [2]:
import arxiv
import json
import os
from typing import List
from dotenv import load_dotenv

## Tool Functions

In [3]:
PAPER_DIR = "papers"

The first tool searches for relevant arXiv papers based on a topic and stores the papers' info in a JSON file (title, authors, summary, paper url and the publication date). The JSON files are organized by topics in the `papers` directory. The tool does not download the papers.  

In [4]:
# Use arxiv to find the papers 
client = arxiv.Client()

# Search for the most relevant articles matching the queried topic
search = arxiv.Search(
    query = "computers",
    max_results = 5,
    sort_by = arxiv.SortCriterion.Relevance
)
search

arxiv.Search(query='computers', id_list=[], max_results=5, sort_by=<SortCriterion.Relevance: 'relevance'>, sort_order=<SortOrder.Descending: 'descending'>)

In [5]:
papers = client.results(search)
list(papers)

[arxiv.Result(entry_id='http://arxiv.org/abs/1310.7911v2', updated=datetime.datetime(2013, 12, 9, 21, 56, 24, tzinfo=datetime.timezone.utc), published=datetime.datetime(2013, 10, 29, 18, 29, 13, tzinfo=datetime.timezone.utc), title='Compact manifolds with computable boundaries', authors=[arxiv.Result.Author('Zvonko Iljazovic')], summary='We investigate conditions under which a co-computably enumerable closed set\nin a computable metric space is computable and prove that in each locally\ncomputable computable metric space each co-computably enumerable compact\nmanifold with computable boundary is computable. In fact, we examine the notion\nof a semi-computable compact set and we prove a more general result: in any\ncomputable metric space each semi-computable compact manifold with computable\nboundary is computable. In particular, each semi-computable compact\n(boundaryless) manifold is computable.', comment=None, journal_ref='Logical Methods in Computer Science, Volume 9, Issue 4 (Dece

In [6]:
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    Search for papers on arXiv based on a topic and store their information.
    
    Args:
        topic: The topic to search for
        max_results: Maximum number of results to retrieve (default: 5)
        
    Returns:
        List of paper IDs found in the search
    """

    # Use arxiv to find the papers 
    client = arxiv.Client()

    # Search for the most relevant articles matching the queried topic
    search = arxiv.Search(
        query = topic,
        max_results = max_results,
        sort_by = arxiv.SortCriterion.Relevance
    )

    papers = client.results(search)

    # Create directory for this topic
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)

    file_path = os.path.join(path, "papers_info.json")

    # Try to load existing papers info
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # Process each paper and add to papers_info  
    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info

    # Save updated papers_info to json file
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)

    print(f"Results are saved in: {file_path}")

    return paper_ids

In [7]:
search_papers("computers")

Results are saved in: papers/computers/papers_info.json


['1310.7911v2',
 'math/9711204v1',
 '2208.00733v1',
 '2504.07020v1',
 '2403.03925v1']

The second tool looks for information about a specific paper across all topic directories inside the `papers` directory.

In [8]:
def extract_info(paper_id: str) -> str:
    """
    Search for information about a specific paper across all topic directories.
    
    Args:
        paper_id: The ID of the paper to look for
        
    Returns:
        JSON string with paper information if found, error message if not found
    """

    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue

    return f"There's no saved information related to paper {paper_id}."

In [9]:
extract_info('1310.7911v2')

'{\n  "title": "Compact manifolds with computable boundaries",\n  "authors": [\n    "Zvonko Iljazovic"\n  ],\n  "summary": "We investigate conditions under which a co-computably enumerable closed set\\nin a computable metric space is computable and prove that in each locally\\ncomputable computable metric space each co-computably enumerable compact\\nmanifold with computable boundary is computable. In fact, we examine the notion\\nof a semi-computable compact set and we prove a more general result: in any\\ncomputable metric space each semi-computable compact manifold with computable\\nboundary is computable. In particular, each semi-computable compact\\n(boundaryless) manifold is computable.",\n  "pdf_url": "http://arxiv.org/pdf/1310.7911v2",\n  "published": "2013-10-29"\n}'

## Tool Schema

Here are the schema of each tool which you will provide to the LLM.

In [10]:
tools = [
    {   "type": "function",
        "function": {
            "name": "search_papers",
            "description": "Search for papers on arXiv based on a topic and store their information.",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "The topic to search for"
                    },
                    "max_results": {
                        "type": "integer",
                        "description": "Maximum number of results to retrieve",
                        "default": 5
                    }
                },
                "required": ["topic"]
            }
        }
    },
    {   "type": "function",
        "function": {
            "name": "extract_info",
            "description": "Search for information about a specific paper across all topic directories.",
            "parameters": {
                "type": "object",
                "properties": {
                    "paper_id": {
                        "type": "string",
                        "description": "The ID of the paper to look for"
                    }
                },
                "required": ["paper_id"]
            }
        }
    }
]

## Tool Mapping

This code handles tool mapping and execution.

In [11]:
mapping_tool_function = {
    "search_papers": search_papers,
    "extract_info": extract_info
}

def execute_tool(tool_name, tool_args):

    result = mapping_tool_function[tool_name](**tool_args)

    if result is None:
        result = "The operation completed but didn't return any results."

    elif isinstance(result, list):
        result = ', '.join(result)

    elif isinstance(result, dict):
        # Convert dictionaries to formatted JSON strings
        result = json.dumps(result, indent=2)

    else:
        # For any other type, convert using str()
        result = str(result)
    return result

## Chatbot Code

The chatbot handles the user's queries one by one, but it does not persist memory across the queries.

In [12]:
load_dotenv()
from openai import OpenAI
client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

### Query Processing

In [17]:
def function_calling(messages):
    completion = client.chat.completions.create(
        max_tokens = 2024,
        model="qwen-plus",  # 此处以qwen-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
        messages=messages,
        tools=tools
    )
    return completion

def process_query(query):

    messages = [{'role': 'user', 'content': query}]

    process_query = True
    while process_query:
        completion = function_calling(messages)
        if completion.choices[0].message.content:
            print(completion.choices[0].message.content)
            process_query = False

        elif len(completion.choices[0].message.tool_calls) > 0:
            tool_calls = completion.choices[0].message.tool_calls
            tool_name = tool_calls[0].function.name
            tool_args = tool_calls[0].function.arguments
            tool_args = json.loads(tool_args)
            print(f"Calling tool {tool_name} with args {tool_args}")
            result = execute_tool(tool_name, tool_args)
            messages.append(completion.choices[0].message)
            messages.append({"role": "tool", "content": result, "tool_call_id": tool_calls[0].id})

### Chat Loop

In [18]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("\nQuery: ").strip()
            if query.lower() == 'quit':
                break

            process_query(query)
            print("\n")
        except Exception as e:
            print(f"\nError: {str(e)}")

Feel free to interact with the chatbot. Here's an example query: 

- Search for 2 papers on "LLM interpretability"

To access the `papers` folder: 1) click on the `File` option on the top menu of the notebook and 2) click on `Open` and then 3) click on `L3`.

In [19]:
chat_loop()

Type your queries or 'quit' to exit.
Calling tool search_papers with args {'topic': 'LLM interpretability', 'max_results': 2}
Results are saved in: papers/llm_interpretability/papers_info.json
I found two papers on "LLM interpretability". Here are their IDs:

1. Paper ID: 2412.07992v3
2. Paper ID: 2402.01761v1

If you need more information about these papers, please let me know!


It seems like you might be asking for a summary of two specific papers, but your request is a bit unclear. Could you please clarify which two papers you would like summarized? If you have the IDs or topics of the papers in question, please provide them. This will help me give you the information you're looking for. If you want, I can search for papers based on a topic or extract information about specific papers by their ID. What would you like to do?


Calling tool extract_info with args {'paper_id': '2412.07992v3'}
Calling tool extract_info with args {'paper_id': '2402.01761v1'}
Here are the details for the

In [20]:
chat_loop()

Type your queries or 'quit' to exit.
Hello! How can I assist you today?


I currently have two tools at my disposal:

1. `search_papers`: This function allows me to search for papers on arXiv based on a specified topic and then store their information. It requires the input parameters such as the topic to search for and the maximum number of results to retrieve (with a default of 5).

2. `extract_info`: This function is used to search for information about a specific paper across all topic directories. It requires the input parameter of the paper ID to look for.

Would you like to use either of these functions to find a paper or extract information about a specific one? If so, please provide me with the necessary details!




## Resources

[Guide on how to implement tool use](https://docs.anthropic.com/en/docs/build-with-claude/tool-use/overview#how-to-implement-tool-use)